In [6]:
import pyTigerGraph as tg
import pandas as pd
import json 
import gradio as gr
hostName = "https://eaa8afeab8334e1b9f6d6b95b6216f59.i.tgcloud.io"
graphName = "VWG"
secret = "n0r4908rshcutm9vda0qb8c77iep27j0"
userName = "user_1"
password = "A1z2e3r4*"

graph = tg.TigerGraphConnection(host=hostName, graphname=graphName)

authToken = graph.getToken(secret)
authToken = authToken[0]
print(f"secret token: {authToken}")
conn = tg.TigerGraphConnection(host=hostName, graphname=graphName, username=userName, password=password, apiToken=authToken)

secret token: h59smdtkr2uuci5n57eal62qbic1buts


In [9]:
node_final = "Bilbao"
node_initial = "Madrid"
nodetype = "Nodes"
edgetype = "distribute_to"
weight_attribute = "Capacity"


results = conn.runInstalledQuery("tg_astar_test",params={"source_vertex": node_final, "source_vertex.type" : nodetype,
                                                         "target_vertex" : node_initial, "target_vertex.type" : nodetype,
                                                     "e_type_set": edgetype,"weight_type" : "FLOAT",
                                                     "latitude" : "latitude", "longitude" : "longitude",
                                                     "weight_attribute": weight_attribute,
                                                     "print_stats":"True"})

In [10]:
results

[{'@@sum_total_dist': 40},
 {'hop': 1},
 {'@@display_edge_set': [{'e_type': 'distribute_to',
    'from_id': 'Madrid',
    'from_type': 'Nodes',
    'to_id': 'Bilbao',
    'to_type': 'Nodes',
    'directed': False,
    'attributes': {'Daily_movement': 0, 'Price': 6, 'Capacity': 40}}],
  'tmp': [{'v_id': 'Bilbao',
    'v_type': 'Nodes',
    'attributes': {'Capacity': 17,
     'Stock': 15,
     'LoadCapacity': 500,
     'UnloadCapacity': 4,
     'latitude': 43.263,
     'longitude': -2.934,
     '@min_dist_heap': [{'dist': 0, 'v': 'Bilbao'}],
     '@or_visited': True}},
   {'v_id': 'Madrid',
    'v_type': 'Nodes',
    'attributes': {'Capacity': 117,
     'Stock': 106,
     'LoadCapacity': 500,
     'UnloadCapacity': 15,
     'latitude': 40.4168,
     'longitude': -3.7038,
     '@min_dist_heap': [{'dist': 40, 'v': 'Bilbao'}],
     '@or_visited': True}}]}]

In [12]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Lire le CSV entier
df = pd.read_csv(r'C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\df_light.csv')

# Convertir les lignes en une liste de dictionnaires
data_list = df.to_dict(orient='records')

In [15]:
data_list

[{'ID': 178,
  'CODE_Origin': 'Madrid',
  'CODE_Destination': 'Bilbao',
  'Palets': 1},
 {'ID': 180,
  'CODE_Origin': 'Madrid',
  'CODE_Destination': 'Santander',
  'Palets': 2},
 {'ID': 181,
  'CODE_Origin': 'Malaga',
  'CODE_Destination': 'Santander',
  'Palets': 10}]

In [16]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Variables constantes pour le type de nœud, le type d'arête et l'attribut de poids
nodetype = "Nodes"
edgetype = "distribute_to"
weight_attribute = "Capacity"

# Fonction pour envoyer la requête
def send_request(row):
    try:
        # Extraire les paramètres de la ligne de données
        node_final = row['CODE_Destination']
        node_initial = row['CODE_Origin']
        
        params = {
            "source_vertex": node_final,
            "source_vertex.type": nodetype,
            "target_vertex": node_initial,
            "target_vertex.type": nodetype,
            "e_type_set": edgetype,
            "weight_type": "FLOAT",
            "latitude": "latitude",
            "longitude": "longitude",
            "weight_attribute": weight_attribute,
            "print_stats": "True"
        }

        # Envoyer la requête à l'algorithme dans le cluster cloud
        results = conn.runInstalledQuery("tg_astar_test", params=params)
        return results
    except Exception as e:
        return f"Exception: {str(e)}"

# Utiliser ThreadPoolExecutor pour envoyer les requêtes en parallèle
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_row = {executor.submit(send_request, row): row for row in data_list}
    for future in as_completed(future_to_row):
        row = future_to_row[future]
        try:
            response = future.result()
            print(f"Réponse pour ID {row['ID']} ({row['CODE_Origin']} -> {row['CODE_Destination']}): {response}")
        except Exception as exc:
            print(f"Erreur pour ID {row['ID']} ({row['CODE_Origin']} -> {row['CODE_Destination']}): {exc}")


Réponse pour ID 178 (Madrid -> Bilbao): [{'@@sum_total_dist': 40}, {'hop': 1}, {'@@display_edge_set': [{'e_type': 'distribute_to', 'from_id': 'Madrid', 'from_type': 'Nodes', 'to_id': 'Bilbao', 'to_type': 'Nodes', 'directed': False, 'attributes': {'Daily_movement': 0, 'Price': 6, 'Capacity': 40}}], 'tmp': [{'v_id': 'Bilbao', 'v_type': 'Nodes', 'attributes': {'Capacity': 17, 'Stock': 15, 'LoadCapacity': 500, 'UnloadCapacity': 4, 'latitude': 43.263, 'longitude': -2.934, '@min_dist_heap': [{'dist': 0, 'v': 'Bilbao'}], '@or_visited': True}}, {'v_id': 'Madrid', 'v_type': 'Nodes', 'attributes': {'Capacity': 117, 'Stock': 106, 'LoadCapacity': 500, 'UnloadCapacity': 15, 'latitude': 40.4168, 'longitude': -3.7038, '@min_dist_heap': [{'dist': 40, 'v': 'Bilbao'}], '@or_visited': True}}]}]
Réponse pour ID 181 (Malaga -> Santander): [{'@@sum_total_dist': 80}, {'hop': 4}, {'@@display_edge_set': [{'e_type': 'distribute_to', 'from_id': 'Bilbao', 'from_type': 'Nodes', 'to_id': 'Santander', 'to_type': 'No

In [18]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Lire le CSV entier
df = pd.read_csv(r'C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\df_light.csv')

# Vérifier les colonnes du DataFrame pour s'assurer qu'elles correspondent aux noms attendus
print(df.columns)

# Convertir les lignes en une liste de dictionnaires
data_list = df.to_dict(orient='records')

# Variables globales pour suivre les opérations
operation_count = 0
palets_number = 0
palets_cost = 0
unique_origin_nodes = set()
unique_destination_nodes = set()
log_list = []


def path_taken(results):
    edges = results[2]['@@display_edge_set']

    # initialise the initial node
    node_initial = None
    # find initial node
    edge_to_ids = {edge['to_id'] for edge in edges}
    for edge in edges:
        if edge['from_id'] not in edge_to_ids:
            node_initial = edge['from_id']
            break

    if node_initial is None:
        raise ValueError("initial node cannot be determined.")

    # initialise the current node
    current_node = node_initial

    # list to store the path
    path = [current_node]

    # iterate through the edges to find the next node
    while True:
        next_node = None
        for edge in edges:
            if edge['from_id'] == current_node:
                next_node = edge['to_id']
                break
        if next_node is None:
            break
        path.append(next_node)
        current_node = next_node

    # Construct the dictionary of nodes
    nodes = {node_name: node_name for node_name in path}

    # construct list of ordered nodes
    ordered_nodes = [nodes[node_name] for node_name in path]

    # return list of ordered nodes
    return ordered_nodes

# Définition de la fonction de transfert (doit être définie en dehors de la fonction send_request)
def transfert_nodes_and_edges(node_initial, node_final, charge, weight_attribute="Capacity", nodetype="Nodes", edgetype="distribute_to"):
    global operation_count, palets_number, palets_cost, unique_origin_nodes, unique_destination_nodes, log_list

    results = conn.runInstalledQuery("tg_astar_test", params={"source_vertex": node_final, "source_vertex.type": nodetype,
                                                              "target_vertex": node_initial, "target_vertex.type": nodetype,
                                                              "e_type_set": edgetype, "weight_type": "FLOAT",
                                                              "latitude": "latitude", "longitude": "longitude",
                                                              "weight_attribute": weight_attribute,
                                                              "print_stats": "True"})

    order_taken = path_taken(results)  # Order the path taken (put the nodes in order) (format: [namenode1, namenode2, ...])

    # Extract edges data and put it in the order of delivery
    edge_set = results[2]['@@display_edge_set']
    reordered_edges_set = []
    for location in order_taken:
        for edge in edge_set:
            if edge['from_id'] == location:
                reordered_edges_set.append(edge)

    # Extract nodes data and put it in the order of delivery
    node_set = results[2]['tmp']
    reordered_nodes_set = []
    for location in order_taken:
        for item in node_set:
            if item['v_id'] == location:
                reordered_nodes_set.append(item)

    try:
        node_i_stock = reordered_nodes_set[0]["attributes"]["Stock"]
        node_f_stock = reordered_nodes_set[-1]["attributes"]["Stock"]
        node_i_unload_capacity = reordered_nodes_set[0]["attributes"]["UnloadCapacity"]
        node_f_load_capacity = reordered_nodes_set[-1]["attributes"]["LoadCapacity"]
        # Nodes constraint
        if charge > node_i_stock:
            error_message = f"{node_initial} to {node_final}: Not enough stock ({charge} > {node_i_stock})"
            log_list.append(error_message)
            return error_message

        if charge > node_f_load_capacity:
            error_message = f"{node_initial} to {node_final}: Not enough load capacity from receiving warehouse (charge: {charge} > LoadCapacity: {node_f_load_capacity})"
            log_list.append(error_message)
            return error_message

        if charge > node_i_unload_capacity:
            error_message = f"{node_initial} to {node_final}: Not enough unload capacity from starting warehouse (charge: {charge} > UnloadCapacity: {node_i_unload_capacity})"
            log_list.append(error_message)
            return error_message

        capacity_edge = reordered_edges_set[0]["attributes"]["Capacity"]
        daily_movement = reordered_edges_set[0]["attributes"]["Daily_movement"]
        # Edge constraint
        if daily_movement + charge > capacity_edge:
            error_message = f"Send {capacity_edge} from {node_initial} to {node_final}: Not enough edge capacity ({daily_movement + charge} > {capacity_edge})"
            log_list.append(error_message)
            return error_message

        # Update Nodes Stock data
        nodes_batch = [
            (node_initial, {"Stock": node_i_stock - charge}),
            (node_final, {"Stock": node_f_stock + charge})
        ]
        conn.upsertVertices("Nodes", nodes_batch)

        # Update Edges data (daily movements)
        edges_batch = []
        for element in reordered_edges_set:
            element['attributes']['Daily_movement'] += charge
            edges_batch.append((element["from_id"], element["to_id"], {"Daily_movement": element["attributes"]["Daily_movement"]}))
            palets_cost += element["attributes"]["Price"]

        conn.upsertEdges(sourceVertexType=nodetype,
                         targetVertexType=nodetype,
                         edgeType=edgetype,
                         edges=edges_batch)

        unique_origin_nodes.add(node_initial)
        unique_destination_nodes.add(node_final)

        operation_count += 1  # increment operation_count
        palets_number += charge  # increment palets_number with charge

    except Exception as e:
        error_message = f"Error in the transfer: {e}"
        log_list.append(error_message)
        return error_message

# Fonction pour envoyer la requête et traiter les données
def send_request(row):
    try:
        node_initial = row['CODE_Origin']
        node_final = row['CODE_Destination']
        charge = row['Palets']
        
        result = transfert_nodes_and_edges(node_initial, node_final, charge)
        return result
    except Exception as e:
        return f"Exception: {str(e)}"

# Utiliser ThreadPoolExecutor pour envoyer les requêtes en parallèle
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_row = {executor.submit(send_request, row): row for row in data_list}
    for future in as_completed(future_to_row):
        row = future_to_row[future]
        try:
            response = future.result()
            print(f"Réponse pour ID {row['ID']} ({row['CODE_Origin']} -> {row['CODE_Destination']}): {response}")
        except Exception as exc:
            print(f"Erreur pour ID {row['ID']} ({row['CODE_Origin']} -> {row['CODE_Destination']}): {exc}")

# Afficher le résumé des opérations
print(f"Total des opérations: {operation_count}")
print(f"Nombre total de palets déplacés: {palets_number}")
print(f"Coût total des palets: {palets_cost}")
print(f"Nœuds d'origine uniques: {unique_origin_nodes}")
print(f"Nœuds de destination uniques: {unique_destination_nodes}")
print(f"Log des erreurs: {log_list}")



Index(['ID', 'CODE_Origin', 'CODE_Destination', 'Palets'], dtype='object')
Réponse pour ID 181 (Malaga -> Santander): Malaga to Santander: Not enough unload capacity from starting warehouse (charge: 10 > UnloadCapacity: 8)
Réponse pour ID 178 (Madrid -> Bilbao): None
Réponse pour ID 180 (Madrid -> Santander): None
Total des opérations: 2
Nombre total de palets déplacés: 3
Coût total des palets: 18
Nœuds d'origine uniques: {'Madrid'}
Nœuds de destination uniques: {'Santander', 'Bilbao'}
Log des erreurs: ['Malaga to Santander: Not enough unload capacity from starting warehouse (charge: 10 > UnloadCapacity: 8)']


In [40]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# # Fonction pour envoyer la requête et traiter les données
# def send_request(row):
#     try:
#         node_initial = row['CODE_Origin']
#         node_final = row['CODE_Destination']
#         charge = row['Palets']
        
#         result = transfert_nodes_and_edges(node_initial, node_final, charge)
#         return result
#     except Exception as e:
#         return f"Exception: {str(e)}"


# Définition de la fonction de transfert (doit être définie en dehors de la fonction send_request)
def transfert_nodes_and_edges(node_initial, node_final, charge, weight_attribute="Capacity", nodetype="Nodes", edgetype="distribute_to"):
    global operation_count, palets_number, palets_cost, unique_origin_nodes, unique_destination_nodes, log_list

    results = conn.runInstalledQuery("tg_astar_test", params={"source_vertex": node_final, "source_vertex.type": nodetype,
                                                              "target_vertex": node_initial, "target_vertex.type": nodetype,
                                                              "e_type_set": edgetype, "weight_type": "FLOAT",
                                                              "latitude": "latitude", "longitude": "longitude",
                                                              "weight_attribute": weight_attribute,
                                                              "print_stats": "True"})

    order_taken = path_taken(results)  # Order the path taken (put the nodes in order) (format: [namenode1, namenode2, ...])

    # Extract edges data and put it in the order of delivery
    edge_set = results[2]['@@display_edge_set']
    reordered_edges_set = []
    for location in order_taken:
        for edge in edge_set:
            if edge['from_id'] == location:
                reordered_edges_set.append(edge)

    # Extract nodes data and put it in the order of delivery
    node_set = results[2]['tmp']
    reordered_nodes_set = []
    for location in order_taken:
        for item in node_set:
            if item['v_id'] == location:
                reordered_nodes_set.append(item)

    try:
        node_i_stock = reordered_nodes_set[0]["attributes"]["Stock"]
        node_f_stock = reordered_nodes_set[-1]["attributes"]["Stock"]
        node_i_unload_capacity = reordered_nodes_set[0]["attributes"]["UnloadCapacity"]
        node_f_load_capacity = reordered_nodes_set[-1]["attributes"]["LoadCapacity"]
        # Nodes constraint
        if charge > node_i_stock:
            error_message = f"{node_initial} to {node_final}: Not enough stock ({charge} > {node_i_stock})"
            log_list.append(error_message)
            # return error_message

        if charge > node_f_load_capacity:
            error_message = f"{node_initial} to {node_final}: Not enough load capacity from receiving warehouse (charge: {charge} > LoadCapacity: {node_f_load_capacity})"
            log_list.append(error_message)
            # return error_message

        if charge > node_i_unload_capacity:
            error_message = f"{node_initial} to {node_final}: Not enough unload capacity from starting warehouse (charge: {charge} > UnloadCapacity: {node_i_unload_capacity})"
            log_list.append(error_message)
            # return error_message

        capacity_edge = reordered_edges_set[0]["attributes"]["Capacity"]
        daily_movement = reordered_edges_set[0]["attributes"]["Daily_movement"]
        # Edge constraint
        if daily_movement + charge > capacity_edge:
            error_message = f"Send {capacity_edge} from {node_initial} to {node_final}: Not enough edge capacity ({daily_movement + charge} > {capacity_edge})"
            log_list.append(error_message)
            # return error_message

        # Update Nodes Stock data
        nodes_batch = [
            (node_initial, {"Stock": node_i_stock - charge}),
            (node_final, {"Stock": node_f_stock + charge})
        ]
        print("send vertices")
        conn.upsertVertices("Nodes", nodes_batch)
        print("vertices sent")

        # Update Edges data (daily movements)
        edges_batch = []
        for element in reordered_edges_set:
            element['attributes']['Daily_movement'] += charge
            edges_batch.append((element["from_id"], element["to_id"], {"Daily_movement": element["attributes"]["Daily_movement"]}))
            palets_cost += element["attributes"]["Price"]
        print("send edges")
        conn.upsertEdges(sourceVertexType=nodetype,
                         targetVertexType=nodetype,
                         edgeType=edgetype,
                         edges=edges_batch)
        print("edges sent")
        unique_origin_nodes.add(node_initial)
        unique_destination_nodes.add(node_final)

        operation_count += 1  # increment operation_count
        palets_number += charge  # increment palets_number with charge

    except Exception as e:
        error_message = f"Error in the transfer: {e}"
        log_list.append(error_message)
        return error_message

# # Fonction pour effectuer les mouvements quotidiens
# def make_daily_movements(input_file, column_origin, column_destination, column_transfert):
#     global operation_count, unique_origin_nodes, palets_number, palets_cost, unique_destination_nodes, log_list
#     operation_count = 0
#     palets_number = 0
#     palets_cost = 0
#     unique_origin_nodes = set()
#     unique_destination_nodes = set()

#     log_list = []

#     df = pd.read_csv(input_file)

#     for index, row in df.iterrows():
#         origin = str(row[column_origin])
#         destination = str(row[column_destination])
#         volume = float(row[column_transfert])

#         try:
#             transfert_nodes_and_edges(origin, destination, volume)

#         except Exception as e:
#             log_list.append(f"Error in make_daily_movements: {e}")
#             continue

#     return '\n'.join(log_list)

In [41]:
# # Utiliser ThreadPoolExecutor pour envoyer les requêtes en parallèle
# with ThreadPoolExecutor(max_workers=10) as executor:
#     future_to_row = {executor.submit(send_request, row): row for row in data_list}
#     for future in as_completed(future_to_row):
#         row = future_to_row[future]
#         try:
#             response = future.result()
#             print(f"Réponse pour ID {row['ID']} ({row['CODE_Origin']} -> {row['CODE_Destination']}): {response}")
#         except Exception as exc:
#             print(f"Erreur pour ID {row['ID']} ({row['CODE_Origin']} -> {row['CODE_Destination']}): {exc}")

# # Afficher le résumé des opérations
# print(f"Total des opérations: {operation_count}")
# print(f"Nombre total de palets déplacés: {palets_number}")
# print(f"Coût total des palets: {palets_cost}")
# print(f"Nœuds d'origine uniques: {unique_origin_nodes}")
# print(f"Nœuds de destination uniques: {unique_destination_nodes}")
# print(f"Log des erreurs: {log_list}")


In [42]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def make_daily_movements(input_file, column_origin, column_destination, column_transfert):
    global operation_count, unique_origin_nodes, palets_number, palets_cost, unique_destination_nodes, log_list
    operation_count = 0
    palets_number = 0
    palets_cost = 0
    unique_origin_nodes = set()
    unique_destination_nodes = set()
    log_list = []

    df = pd.read_csv(input_file)

    # Fonction pour effectuer un transfert de palets et gérer les exceptions
    def perform_transfer(row):
        try:
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            volume = float(row[column_transfert])

            transfert_nodes_and_edges(origin, destination, volume)

        except Exception as e:
            log_list.append(f"Error in make_daily_movements: {e}")

    # Utilisation de ThreadPoolExecutor pour paralléliser les transferts
    with ThreadPoolExecutor(max_workers=8) as executor:
        # Création des futures pour chaque ligne du DataFrame
        futures = [executor.submit(perform_transfer, row) for index, row in df.iterrows()]

        # Attendre que toutes les tâches soient terminées
        for future in as_completed(futures):
            pass

    # Retourner les messages d'erreur concaténés
    return '\n'.join(log_list)


In [50]:
log_messages = make_daily_movements(r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\df_light_100.csv", "CODE_Origin", "CODE_Destination", "Palets")

print(log_messages)

send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
send vertices
vertices sentvertices sent
send edges
vertices s